In [19]:
from pandas import Timestamp
import numpy as np
from numba import njit
import numba
import pycurl_requests as requests
import pandas as pd
from pprint import pprint
import sys
import itertools

sys.path.append(
    "/Users/GlebSokolov/.vscode/extensions/almenon.arepl-2.0.3/node_modules/arepl-backend/python"
)
# from arepl_dump import dump
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [12]:
import coinbase
from coinbase.wallet.model import APIObject
from coinbase.wallet.client import Client
import cbpro

coinbase_API_key = "PZsUB9TdWedNjJcu"
coinbase_API_secret = "UP8Jz2cfXvDRdgeeNwFy6fCxfJCkQFea"
client = Client(coinbase_API_key, coinbase_API_secret)


In [22]:
import time
import numpy as np
import ccxt

symbol = "AGLD-USDT"

exchange = getattr(ccxt, "coinbase")()
since = "2021-10-05 10:00:00"
since_ts = exchange.parse8601(since)
end_ts = since_ts + (3600000)
end = str(pd.to_datetime(end_ts, unit="ms"))


In [14]:
%%capture
client.get_spot_price(currency='AGLD')
client._make_api_object(client._get('v2', 'prices', 'AGLD-USD', 'historic'), APIObject)

In [ ]:
%%capture
public_client = cbpro.PublicClient()
public_client.get_product_ticker(product_id='AGLD-USD')
responses = list(public_client.get_product_trades(product_id=symbol)
)

In [ ]:
# %%snoop
trade_ids = np.zeros(len(responses), dtype=int)
rrr = np.zeros(len(responses), dtype=np.int64)
for i in range(rrr.shape[0]):
    rrr[i] = int(Timestamp(responses[i]["time"]).timestamp() * 1000)
    trade_ids[i] = responses[i]["trade_id"]


In [ ]:
@njit
def findid(lst, ts_start, ts_end, trade_ids):
    ids = []
    for i in range((len(lst))):
        if ts_start <= lst[i] <= ts_end:
            ids.append((trade_ids[i], i))
    return np.array(ids)


In [ ]:
data = public_client.get_product_historic_rates(
    start=since, end=end, product_id="AGLD-USDT"
)
data = pd.DataFrame(data, columns=["time", "low", "high", "open", "close", "volume"])
data["time_iso"] = pd.to_datetime(data["time"], unit="s")


In [27]:
%%script false --no-raise-error
headers = {
    "User-Agent": "python-requests/2.26.0",
    "Accept": "*/*",
    "Connection": "keep-alive",
}
responses = [
    requests.get(
        f"https://api.exchange.coinbase.com/products/AGLD-USDT/trades?limit=1000&after={i*1000}",
        headers=headers,
    ).json()
    for i in range(70)
]
responses.pop(0)

In [26]:
%%script false --no-raise-error
# dir(public_client)

object_methods = [
    method_name
    for method_name in dir(public_client)
    if callable(getattr(public_client, method_name))
]
pprint(object_methods)
